#Introdução

Recebido o arquivo SDW2023.csv com as IDs dos clientes que precisam receber mensagens de marketing de forma personalizada, verificamos que poderia ser utilizado IA generativa para personalizar essas mensagens.
Com base nessa lista de IDs, será consumido o endpoint GET https://sdw-2023-prd.up.railway.app/users/{id} GET (API da Santander Dev Week 2023) para obter os dados de cada cliente.
As mensagens estando prontas, serão submentidas para atualização da base de dados utilizando o endpoint PUT https://sdw-2023-prd.up.railway.app/users/{id}.

Link da API: https://sdw-2023-prd.up.railway.app/swagger-ui/index.html#/

In [ ]:
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

#Extração de dados

In [ ]:
# Importando a biblioteca pandas
import pandas as pd

# Criando o dataframe a partir do arquivo recebido
df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[1, 2, 3, 4, 5]


In [ ]:
# Importando as bibliotecas requests e json
import requests
import json

# Criando a função para utilizar o método GET
def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

#Transformação

In [ ]:
# Instalando a API da OpenAI GPT-4
!pip install openai

In [ ]:
# Definindo a key
openai_api_key = 'TODO'

In [ ]:
# Importando a biblioteca da OpenAI
import openai

openai.api_key = openai_api_key

# Criando função para gerar textos de marketing personalizados
def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

# Loop de criação de textos de acordo com cada cliente da lista
for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

#Carregamento

In [ ]:
# Definindo função para atualizar os registros dos usuários
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

# Criando loop para atualizar todos os usuários da lista
for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")